In [1]:
import requests
import json
import time
from bs4 import BeautifulSoup
import pandas as pd
import datetime

import plotly.express as px
import plotly.graph_objects as go

# コンテスト情報

In [3]:
problem_id = "5007"

contest_start = datetime.datetime(2023,4,23,19,0,0)
contest_end = datetime.datetime(2023,4,30,19,0,0)

# アルゴ、ヒューリスティックの順番
member_rate = {'comavius': (862, 471),
 'emi1': (347, 0),
 'eoeo': (1117, 305),
 'Michirakara': (1090, 930),
 'soso': (1089, 150),
 'tmikada': (579, 871),
 'jakujaku12': (1361, 1551),
 'yas_yasyu': (946, 1035),
 'Edomonndo365': (809, 1110),
 'ゆきひら鍋': (1367, 948),
 'dna4_': (1325, 1428),
 'tanaka': (1494, 1574),
 'hiraku': (1272, 1470),
 'lud404Qe8lItbFm': (1085, 1051),
 'moharan627': (1292, 863),
 'r3yohei': (708, 1327),
 'fujikawahiroaki': (598, 1342),
 'hotpepsi': (1627, 1593),
 'keroru': (1607, 1401),
 'motoshira': (1429, 1344),
 'ococonomy1': (1678, 1244),
 'nephrologist': (1618, 1420),
 '37kt': (2070, 1229),
 'bird01': (1334, 1506),
 'brthyyjp': (1668, 1253),
 'prussian_coder': (2139, 1398),
 'yamake': (2066, 1363),
 'ながたかな': (2263, 969),
 'highjump': (1411, 2438),
 'komori3': (1634, 2399),
 'ssaattoo': (1435, 2017),
 'tabae326': (2014, 1705),
 'Jiro_tech15': (2411, 2756),
 'ainem-m': (941, 1441)
}

team_members = {
    "クリエイティブフォース": ["comavius", "emi1", "eoeo", "Michirakara", "soso", "tmikada"],
    "writer解を超え隊": ["jakujaku12", "yas_yasyu", "Edomonndo365", "ゆきひら鍋"],
    "エリマキシュリンプキャッツ": ["dna4_", "tanaka", "hiraku", "lud404Qe8lItbFm", "moharan627", "r3yohei"],
    "３代目バベルクライマーズ": ["fujikawahiroaki", "hotpepsi", "keroru", "motoshira", "ococonomy1", "nephrologist"],
    "山魚ブルー": ["37kt", "bird01", "brthyyjp", "prussian_coder", "yamake", "ながたかな"],
    "星のローミィ新作転売ヤー大冒険": ["highjump", "komori3", "ssaattoo", "tabae326", "Jiro_tech15"],
    "運営": ["ainem-m"]
}

team = dict()
for key, members in team_members.items():
  for name in members:
    team[name] = key


In [5]:
#atcoderのIDは別で管理
personal = {'comavius': ['クリエイティブフォース'],
 'emi1y': ['クリエイティブフォース'],
 'eoeo': ['クリエイティブフォース'],
 'Michirakara': ['クリエイティブフォース'],
 'soso15': ['クリエイティブフォース'],
 'tmikada': ['クリエイティブフォース'],
 'jaku': ['writer解を超え隊'],
 'yas_yasyu': ['writer解を超え隊'],
 'edomondo': ['writer解を超え隊'],
 'yukihira_pot': ['writer解を超え隊'],
 'bio4eta': ['エリマキシュリンプキャッツ'],
 'berry': ['エリマキシュリンプキャッツ'],
 'hiraku': ['エリマキシュリンプキャッツ'],
 'hiryuzu': ['エリマキシュリンプキャッツ'],
 'moharan': ['エリマキシュリンプキャッツ'],
 'r3yohei': ['エリマキシュリンプキャッツ'],
 'fujikawahiroaki': ['３代目バベルクライマーズ'],
 'hotpepsi': ['３代目バベルクライマーズ'],
 'keroru': ['３代目バベルクライマーズ'],
 'motoshira': ['３代目バベルクライマーズ'],
 'ococonomy1': ['３代目バベルクライマーズ'],
 'nephrologist': ['３代目バベルクライマーズ'],
 'jin_matakich': ['山魚ブルー'],
 'bird01': ['山魚ブルー'],
 'brthyyjp': ['山魚ブルー'],
 'PrussianBlue': ['山魚ブルー'],
 'yamake': ['山魚ブルー'],
 'ngtkana': ['山魚ブルー'],
 'highjump': ['星のローミィ新作転売ヤー大冒険'],
 'komori3': ['星のローミィ新作転売ヤー大冒険'],
 'ssaattoo': ['星のローミィ新作転売ヤー大冒険'],
 'tabae326': ['星のローミィ新作転売ヤー大冒険'],
 'Jirotech': ['星のローミィ新作転売ヤー大冒険'],
 'ainem': ['運営']}

In [6]:
for name in personal.keys():
  url = "https://atcoder.jp/users/" + name + "/history/json"
  json = requests.session().get(url).json()
  argo_rate = 0
  for data in json:
    ymd, hms_ = data["EndTime"].split("T")
    y,m,d = map(int,ymd.split("-"))
    h,min,sec = map(int,hms_.split("+")[0].split(":"))
    end_time = datetime.datetime(y,m,d,h,min,sec)
    if end_time > contest_start:
      break
    argo_rate = data["NewRating"]

  url = "https://atcoder.jp/users/" + name + "/history/json?contestType=heuristic"
  json = requests.session().get(url).json()
  heur_rate = 0
  heur_highest_perf = 0
  for data in json:
    ymd, hms_ = data["EndTime"].split("T")
    y,m,d = map(int,ymd.split("-"))
    h,min,sec = map(int,hms_.split("+")[0].split(":"))
    end_time = datetime.datetime(y,m,d,h,min,sec)
    if end_time > contest_start:
      break
    heur_rate = data["NewRating"]
    heur_highest_perf = max(heur_highest_perf, data["NewRating"])
  personal[name] += [argo_rate, heur_rate, heur_highest_perf]


In [7]:
df=pd.DataFrame.from_dict(personal, orient="index", columns=["チーム名","Aレート","Hレート","BestPerf"])
df.to_csv("personal_info.csv")
df.head()

,チーム名,Aレート,Hレート,BestPerf
comavius,クリエイティブフォース,862,471,471
emi1y,クリエイティブフォース,347,0,0
eoeo,クリエイティブフォース,1117,305,305
Michirakara,クリエイティブフォース,1090,930,930
soso15,クリエイティブフォース,1089,150,150


# スクレイピング
提出情報を取得する

In [ ]:
def make_datetime(s: str) -> datetime:
  ymd, hms = s.split(" ")
  year, month, day = map(int,ymd.split("-"))
  hour, minute, second = map(int,hms.split(":"))

  res = datetime.datetime(year=year, month=month, day=day, hour=hour, minute=minute, second=second)

  return res

def get_score(submission_id: str) -> int:
  base_url = "https://yukicoder.me/submissions/"
  html = requests.get(base_url + submission_id)
  soup = BeautifulSoup(html.text, "html5lib")
  score = soup.select(".left > table > tbody > tr:nth-child(11) > td:nth-child(2)")[0].text
  return int(score.replace(",",""))

In [ ]:
records = {
    "name": [],
    "team": [],
    "lang": [],
    "score": [],
    "submit": [],
    "elapsed_time": [],
    "argo_rate": [],
    "heuristic_rate": [],
}

not_in_record = set()


page=1
base_url = "https://yukicoder.me/problems/no/" + problem_id + "/submissions?page="
while True:
  standings = requests.get(base_url + str(page))
  soup = BeautifulSoup(standings.text, "html5lib")
  tr = soup.select("tbody > tr")
  if len(tr)==0:
    break
  for row in tr:
    submit = make_datetime(row.contents[3].text)
    result = row.contents[13].text.rsplit()[0]
    if contest_start < submit < contest_end and result=="AC":
      name = row.contents[7].text.rsplit()[0]
      if team.get(name,0)!=0:
        records["name"].append(name)
        records["team"].append(team[name])
        records["lang"].append(row.contents[11].text)
        records["score"].append(get_score(row.contents[1].text))
        records["submit"].append(submit)
        records["elapsed_time"].append(int(row.contents[15].text.rsplit()[0]))
        records["argo_rate"].append(member_rate[name][0])
        records["heuristic_rate"].append(member_rate[name][1])
      else:
        not_in_record.add(name)
    elif submit < contest_start:
      break
  page += 1

if not_in_record: print("バチャ期間中に提出があったが、参加者に登録されていない：", not_in_record)

バチャ期間中に提出があったが、参加者に登録されていない： {'rhoo'}


# データ作成
コンテスト期間中の提出結果から、リストを作成

In [ ]:
df = pd.DataFrame(data=records)
df.columns = ["名前","チーム名", "言語", "スコア", "提出日時", "実行時間", "アルゴレート", "ヒューリスティックレート"]
df.head()

,名前,チーム名,言語,スコア,提出日時,実行時間,アルゴレート,ヒューリスティックレート
0,bird01,山魚ブルー,C++17,8800467,2023-04-30 18:58:27,978,1334,1506
1,fujikawahiroaki,３代目バベルクライマーズ,Crystal,8667014,2023-04-30 18:57:56,954,598,1342
2,brthyyjp,山魚ブルー,PyPy3,8299288,2023-04-30 18:54:29,986,1668,1253
3,motoshira,３代目バベルクライマーズ,Common Lisp,8804192,2023-04-30 18:52:52,916,1429,1344
4,bird01,山魚ブルー,C++17,8801828,2023-04-30 18:51:14,978,1334,1506


## 個人ランキング

In [ ]:
df_individual = df.loc[df.groupby("名前")["スコア"].idxmax()]
df_individual.drop(columns=["言語", "提出日時", "実行時間", "アルゴレート", "ヒューリスティックレート"], inplace=True)
df_individual.sort_values("スコア", ascending=False, inplace=True)
df_individual.reset_index(inplace=True, drop=True)
df_individual["順位"] = range(1,len(df_individual)+1)
df_individual = df_individual.reindex(columns=["順位", "名前", "チーム名", "スコア"])
df_individual.head()

,順位,名前,チーム名,スコア
0,1,tabae326,星のローミィ新作転売ヤー大冒険,9017579
1,2,komori3,星のローミィ新作転売ヤー大冒険,8989314
2,3,highjump,星のローミィ新作転売ヤー大冒険,8953008
3,4,Jiro_tech15,星のローミィ新作転売ヤー大冒険,8895460
4,5,motoshira,３代目バベルクライマーズ,8814582


## チームランキング

In [ ]:
df_team = df.loc[df.groupby("チーム名")["スコア"].idxmax()]
df_team.drop(columns=["言語", "提出日時", "実行時間", "アルゴレート", "ヒューリスティックレート"], inplace=True)
df_team = df_team.reindex(columns = ["チーム名", "名前", "言語", "スコア", "提出日時", "実行時間"])
df_team.sort_values("スコア", ascending=False, inplace=True)
df_team.reset_index(inplace=True, drop=True)
df_team["順位"] = range(1,len(df_team)+1)
df_team = df_team.reindex(columns=["順位", "チーム名", "名前", "スコア"])
df_team.head()

,順位,チーム名,名前,スコア
0,1,星のローミィ新作転売ヤー大冒険,tabae326,9017579
1,2,３代目バベルクライマーズ,motoshira,8814582
2,3,山魚ブルー,bird01,8805587
3,4,クリエイティブフォース,Michirakara,8674763
4,5,エリマキシュリンプキャッツ,r3yohei,8526594


# グラフ作成

1.   縦軸：スコア、横軸：時間の折れ線グラフ（個人）
2.   縦軸：スコア、横軸：時間の折れ線グラフ（チーム）
3.   言語ごとのランキング
4.   スコアとレートの相関関係（アルゴ、ヒューリスティック）




In [ ]:
df_line_indv = df.sort_values("提出日時", ascending = True)
df_line_indv["スコア"] = df_line_indv.groupby("名前")["スコア"].cummax()

fig_individual = px.line(df_line_indv, x="提出日時", y="スコア", color="名前")
fig_individual.show()

In [ ]:
df_line_team = df.sort_values("提出日時", ascending = True)
df_line_team["スコア"] = df_line_team.groupby("チーム名")["スコア"].cummax()

fig_team = px.line(df_line_team, x="提出日時", y="スコア", color="チーム名")
fig_team.show()

In [ ]:
df_lang = df.replace("C++17","C++").replace("C++23(draft)","C++").replace("C++14","C++")
df_lang = df_lang.loc[df_lang.groupby("名前")["スコア"].idxmax()]

fig_lang = px.box(df_lang, x="言語", y="スコア", points='all')
fig_lang.show()

In [ ]:
df_rate = df.loc[df.groupby("名前")["スコア"].idxmax()]

layout = go.Layout(
    xaxis = dict(title="レート", range = [0,3001], dtick=500),   # rangeで範囲、dtick で区間幅
    yaxis = dict(title="スコア（対数）"))

fig_rate = go.Figure(layout = layout)
fig_rate.add_trace(go.Scatter(x = df_rate["アルゴレート"],
                              y = df_rate["スコア"],
                              mode = "markers",
                              name = "アルゴ"))
fig_rate.add_trace(go.Scatter(x = df_rate["ヒューリスティックレート"],
                              y = df_rate["スコア"],
                              mode = "markers",
                              name = "ヒューリスティック"))
fig_rate.update_yaxes(type="log")
fig_rate.show()

## HTMLで書き出し

In [ ]:
fig_individual.write_html("line_individual.html")
fig_team.write_html("line_team.html")
fig_lang.write_html("box_lang.html")
fig_rate.write_html("scatter_rate.html")

In [ ]:
df_individual.to_html(border = 0,
                      classes = ["table", "table-striped", "table-hover"],
                      index = False,
                      justify = "left")

'<table class="dataframe table table-striped table-hover">\n  <thead>\n    <tr style="text-align: left;">\n      <th>順位</th>\n      <th>名前</th>\n      <th>チーム名</th>\n      <th>スコア</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>1</td>\n      <td>tabae326</td>\n      <td>星のローミィ新作転売ヤー大冒険</td>\n      <td>9017579</td>\n    </tr>\n    <tr>\n      <td>2</td>\n      <td>komori3</td>\n      <td>星のローミィ新作転売ヤー大冒険</td>\n      <td>8989314</td>\n    </tr>\n    <tr>\n      <td>3</td>\n      <td>highjump</td>\n      <td>星のローミィ新作転売ヤー大冒険</td>\n      <td>8953008</td>\n    </tr>\n    <tr>\n      <td>4</td>\n      <td>Jiro_tech15</td>\n      <td>星のローミィ新作転売ヤー大冒険</td>\n      <td>8895460</td>\n    </tr>\n    <tr>\n      <td>5</td>\n      <td>motoshira</td>\n      <td>３代目バベルクライマーズ</td>\n      <td>8814582</td>\n    </tr>\n    <tr>\n      <td>6</td>\n      <td>bird01</td>\n      <td>山魚ブルー</td>\n      <td>8805587</td>\n    </tr>\n    <tr>\n      <td>7</td>\n      <td>prussian_coder</td>\n      <td>山魚ブ

In [ ]:
df_team.to_html(border = 0,
                classes = ["table", "table-striped", "table-hover"],
                index = False,
                justify = "left")

'<table class="dataframe table table-striped table-hover">\n  <thead>\n    <tr style="text-align: left;">\n      <th>順位</th>\n      <th>チーム名</th>\n      <th>名前</th>\n      <th>スコア</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>1</td>\n      <td>星のローミィ新作転売ヤー大冒険</td>\n      <td>tabae326</td>\n      <td>9017579</td>\n    </tr>\n    <tr>\n      <td>2</td>\n      <td>３代目バベルクライマーズ</td>\n      <td>motoshira</td>\n      <td>8814582</td>\n    </tr>\n    <tr>\n      <td>3</td>\n      <td>山魚ブルー</td>\n      <td>bird01</td>\n      <td>8805587</td>\n    </tr>\n    <tr>\n      <td>4</td>\n      <td>クリエイティブフォース</td>\n      <td>Michirakara</td>\n      <td>8674763</td>\n    </tr>\n    <tr>\n      <td>5</td>\n      <td>エリマキシュリンプキャッツ</td>\n      <td>r3yohei</td>\n      <td>8526594</td>\n    </tr>\n    <tr>\n      <td>6</td>\n      <td>writer解を超え隊</td>\n      <td>jakujaku12</td>\n      <td>8252884</td>\n    </tr>\n    <tr>\n      <td>7</td>\n      <td>運営</td>\n      <td>ainem-m</td>\n      <td